In [2]:
#Requirements
from langchain_groq import ChatGroq
import streamlit as st
from langchain_core.prompts import ChatPromptTemplate
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


In [3]:
#LLM Model
llama = ChatGroq(
    model="LLaMA3-70B-8192",
    groq_api_key='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',
    temperature=0.0
)

In [5]:
def Assistant():
    base_summary_chain = (
    ChatPromptTemplate.from_template("""
Provide answers for the following list of questions with explapaination and also track which questions are repeated:{topic}
""")
    | llama
    | StrOutputParser()
    |{"base_response": RunnablePassthrough()}
    )
    res =base_summary_chain.invoke("""1. Differentiate Fuzzy logic and Crisp logic. Explain architecture of fuzzy logic system. 
2. Explain various steps of Natural Language Processing.
3. What is state space representation? Explain with suitable example.
4. What is planning? Discuss Nonlinear Planning using Constraint Posting with example.
5. Explain various steps of Natural Language Processing.""")
    return res

In [7]:
print(Assistant())

{'base_response': 'I\'ll provide answers to each question with explanations and track repeated questions.\n\n**1. Differentiate Fuzzy logic and Crisp logic. Explain architecture of fuzzy logic system.**\n\n**Fuzzy Logic vs. Crisp Logic:**\n\nFuzzy logic and crisp logic are two different approaches to reasoning and decision-making.\n\n* **Crisp Logic:** In crisp logic, a statement is either true (1) or false (0). It\'s a binary approach with no intermediate values. Crisp logic is based on classical set theory, where an element either belongs to a set or not.\n* **Fuzzy Logic:** Fuzzy logic, on the other hand, allows for intermediate values between true and false. It\'s based on fuzzy set theory, where an element can partially belong to a set. Fuzzy logic is useful for modeling uncertain or imprecise information.\n\n**Architecture of Fuzzy Logic System:**\n\nThe architecture of a fuzzy logic system consists of the following components:\n\n1. **Fuzzifier:** Converts crisp input values int

In [8]:


# Show title and description.
st.title("💬 Chatbot")
st.write(
    "This is a simple chatbot that uses OpenAI's GPT-3.5 model to generate responses. "
    "To use this app, you need to provide an OpenAI API key, which you can get [here](https://platform.openai.com/account/api-keys). "
    "You can also learn how to build this app step by step by [following our tutorial](https://docs.streamlit.io/develop/tutorials/llms/build-conversational-apps)."
)

# Ask user for their OpenAI API key via `st.text_input`.
# Alternatively, you can store the API key in `./.streamlit/secrets.toml` and access it
# via `st.secrets`, see https://docs.streamlit.io/develop/concepts/connections/secrets-management


# Create a session state variable to store the chat messages. This ensures that the
    # messages persist across reruns.
if "messages" not in st.session_state:
    st.session_state.messages = []

    # Display the existing chat messages via `st.chat_message`.
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    # Create a chat input field to allow the user to enter a message. This will display
    # automatically at the bottom of the page.
    if prompt := st.chat_input("What is up?"):

        # Store and display the current prompt.
        st.session_state.messages.append({"role": "user", "content": prompt})
        with st.chat_message("user"):
            st.markdown(prompt)

        # Generate a response using the OpenAI API.
       

        # Stream the response to the chat using `st.write_stream`, then store it in 
        # session state.
        with st.chat_message("assistant"):
            response = Assistant()
        st.session_state.messages.append({"role": "assistant", "content": response})


2024-10-09 16:39:05.904 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-09 16:39:06.017 
  command:

    streamlit run /Users/parth/Downloads/GithubProjects/AI Student Assistant/AI-Student-Assistant/.venv/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-10-09 16:39:06.017 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-09 16:39:06.018 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-09 16:39:06.018 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-09 16:39:06.018 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-09 16:39:06.019 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored wh